## 1. 使用 tool() 装饰器创建工具

# LangChain 0.3 中的工具（Tools）详解

LangChain 中的工具（Tools）是 AI 系统与外部世界交互的接口，允许模型执行各种操作，如搜索信息、执行计算或调用 API。

## 工具的基本概念

工具由以下几个关键部分组成：
1. **名称**：工具的唯一标识符
2. **描述**：告诉 LLM 如何使用该工具的说明
3. **输入模式**：定义工具接受的输入参数
4. **执行函数**：实际执行操作的代码
5. **返回值处理**：是否将结果直接返回给用户

## 工具的创建方式

### 1. 使用 `tool()` 装饰器创建工具

In [6]:
from langchain_core.tools import tool
import math

@tool
def calculate_square_root(number: float) -> float:
    """计算一个正数的平方根。

    Args:
        number: 需要计算平方根的正数

    Returns:
        输入数字的平方根
    """
    if number < 0:
        raise ValueError("不能计算负数的平方根")
    return math.sqrt(number)

# 使用工具 - 正确方式：传递字典
result = calculate_square_root.invoke({"number": 16.0})
print(f"16的平方根是: {result}")  # 输出: 16的平方根是: 4.0

16的平方根是: 4.0


### 2. 使用 `tool()` 函数创建结构化工具

In [8]:

from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import Optional
import requests

class WeatherInput(BaseModel):
    city: str = Field(..., description="城市名称")
    country: Optional[str] = Field(None, description="国家代码，如CN、US等")

@tool(args_schema=WeatherInput)
def get_weather(city: str, country: Optional[str] = None) -> str:
    """获取指定城市的天气信息。

    Args:
        city: 城市名称
        country: 国家代码（可选）

    Returns:
        天气信息描述
    """
    location = f"{city},{country}" if country else city
    # 这里使用模拟数据，实际应用中应使用真实API
    return f"{location}的天气：晴朗，温度25°C，湿度60%"

# 使用工具
weather_info = get_weather.invoke({"city": "北京", "country": "CN"})
print(weather_info)  # 输出: 北京,CN的天气：晴朗，温度25°C，湿度60%

北京,CN的天气：晴朗，温度25°C，湿度60%



## 工具与模型集成


### 1. 将工具绑定到聊天模型

In [10]:

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# 定义工具
@tool
def multiply(a: int, b: int) -> int:
    """将两个数相乘"""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """将两个数相加"""
    return a + b

# 创建聊天模型
chat_model = ChatOllama(model="qwen3:4b")

# 绑定工具到模型
chat_with_tools = chat_model.bind_tools([multiply, add])

# 使用带工具的模型
messages = [HumanMessage(content="计算 24 乘以 7 是多少？然后再加上 15")]
response = chat_with_tools.invoke(messages)

print("模型回复:", response.content)
print("工具调用:", response.tool_calls if hasattr(response, "tool_calls") else "无工具调用")

模型回复: <think>
好的，用户让我先计算24乘以7，然后再加15。我需要分步骤来处理这个请求。首先，我应该调用乘法函数multiply，参数是24和7。然后，得到结果后，再用加法函数add，把得到的乘积和15相加。

先检查工具里有没有multiply函数。是的，有一个multiply函数，需要a和b两个整数参数。然后，调用这个函数，传入24和7。得到结果后，比如假设乘积是168，然后调用add函数，参数是168和15，这样总和就是183。所以最终结果应该是183。需要确保每一步都正确，没有计算错误。
</think>



工具调用: [{'name': 'multiply', 'args': {'a': 24, 'b': 7}, 'id': 'f3571ad5-be28-49a1-afa5-3e1d91f88b0f', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 168, 'b': 15}, 'id': '5a95dc51-2541-4f2e-acbe-d901686bf46b', 'type': 'tool_call'}]



### 2. 强制模型使用特定工具

In [1]:

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# 定义工具
@tool
def multiply(a: int, b: int) -> int:
    """将两个数相乘"""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """将两个数相加"""
    return a + b

# 创建聊天模型
chat_model = ChatOllama(model="qwen3:0.6b")

# 绑定工具到模型并强制使用multiply工具
chat_with_multiply = chat_model.bind_tools([multiply, add], tool_choice="multiply")

# 使用带工具的模型
messages = [HumanMessage(content="计算 5 加 3 等于多少？")]
response = chat_with_multiply.invoke(messages)

print("模型回复:", response.content)
print("工具调用:", response.tool_calls if hasattr(response, "tool_calls") else "无工具调用")


模型回复: <think>
好的，用户问的是计算5加3等于多少。我需要先确认用户的需求。用户可能是在测试我的功能，或者需要这个简单的加法运算。首先，我需要检查可用的工具，看看有没有相关的函数。在提供的工具中，有一个multiply和add函数，但用户的问题是加法，所以应该使用add函数。参数需要两个整数，这里用户已经给出了5和3，所以直接调用add函数，参数a是5，b是3。不需要其他步骤，直接返回结果即可。
</think>


工具调用: [{'name': 'add', 'args': {'a': 5, 'b': 3}, 'id': '04f27227-5910-4b09-951c-6e3d760dcca7', 'type': 'tool_call'}]



### 3. 工具调用与结果处理

In [3]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 定义工具
@tool
def search_database(query: str) -> str:
    """搜索数据库获取信息"""
    # 模拟数据库搜索
    if "天气" in query:
        return "今天北京晴朗，温度20-25°C"
    elif "新闻" in query:
        return "今日头条：人工智能技术取得重大突破"
    else:
        return "未找到相关信息"

# 创建聊天模型
chat_model = ChatOllama(model="qwen3:0.6b")

# 绑定工具到模型
chat_with_tools = chat_model.bind_tools([search_database])

# 模拟对话流程
messages = [HumanMessage(content="今天北京天气怎么样？")]

# 第一步：模型生成回复，可能包含工具调用
ai_response = chat_with_tools.invoke(messages)
print("AI回复:", ai_response.content)

# 如果有工具调用
if hasattr(ai_response, "tool_calls") and ai_response.tool_calls:
    tool_call = ai_response.tool_calls[0]
    print(f"工具调用: {tool_call['name']}({tool_call['args']})")

    # 执行工具并获取结果
    tool_result = search_database.invoke(tool_call['args']['query'])

    # 将工具结果添加到消息历史
    messages.append(ai_response)
    messages.append(ToolMessage(
        content=tool_result,
        tool_call_id=tool_call['id'],
        name=tool_call['name']
    ))

    # 让模型基于工具结果生成最终回复
    final_response = chat_model.invoke(messages)
    print("最终回复:", final_response.content)

AI回复: <think>
好的，用户问今天北京的天气怎么样。首先，我需要确定用户的需求是什么。他们想知道今天的天气情况，可能需要了解温度、是否有雨或风之类的信息。但根据提供的工具，只有search_database这个函数可以用来获取信息。所以，我应该调用search_database来查询北京今天的天气数据。

接下来，我要确保参数正确。search_database的参数需要一个query，这里应该填写“北京今天天气”。然后，构造一个符合要求的tool_call，参数是{"query": "北京今天天气"}。这样就能返回相关的天气信息了。
</think>


工具调用: search_database({'query': '北京今天天气'})
最终回复: <think>
好的，用户问今天北京的天气，我需要先确认他们的需求。他们可能想知道当前的天气情况，比如温度、天气状况等。根据之前的对话，用户已经提供了天气信息，所以直接回复即可。不过，为了确保信息准确，我应该检查一下是否有其他需要补充的地方。比如，是否需要提到是否有雨或风，或者建议携带雨具之类的。但根据用户提供的信息，回复已经足够，不需要再添加额外内容。保持简洁明了，符合用户的需求。




## 实用工具示例


### 1. 网络搜索工具

In [4]:

from langchain_core.tools import tool
import requests
from bs4 import BeautifulSoup

@tool
def search_web(query: str) -> str:
    """搜索网络获取信息

    Args:
        query: 搜索查询词

    Returns:
        搜索结果摘要
    """
    # 注意：这是一个简化示例，实际应用中应使用适当的搜索API
    try:
        # 模拟搜索结果
        return f"关于'{query}'的搜索结果：\n1. {query}的维基百科页面\n2. 关于{query}的最新新闻\n3. {query}的相关学术研究"
    except Exception as e:
        return f"搜索出错: {str(e)}"

# 使用示例
result = search_web.invoke("量子计算")
print(result)

关于'量子计算'的搜索结果：
1. 量子计算的维基百科页面
2. 关于量子计算的最新新闻
3. 量子计算的相关学术研究



### 2. 文件操作工具

In [ ]:

from langchain_core.tools import tool
import os
import json

@tool
def read_file(file_path: str) -> str:
    """读取文件内容

    Args:
        file_path: 文件路径

    Returns:
        文件内容
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"读取文件出错: {str(e)}"

@tool
def write_file(file_path: str, content: str) -> str:
    """写入内容到文件

    Args:
        file_path: 文件路径
        content: 要写入的内容

    Returns:
        操作结果
    """
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"内容已成功写入到 {file_path}"
    except Exception as e:
        return f"写入文件出错: {str(e)}"

@tool
def list_directory(directory_path: str = '.') -> str:
    """列出目录中的文件和子目录

    Args:
        directory_path: 目录路径，默认为当前目录

    Returns:
        目录内容列表
    """
    try:
        items = os.listdir(directory_path)
        return json.dumps(items, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"列出目录内容出错: {str(e)}"

# 使用示例
# 写入文件
write_result = write_file.invoke({
    "file_path": "test_file.txt",
    "content": "这是一个测试文件内容"
})
print(write_result)

# 读取文件
read_result = read_file.invoke("test_file.txt")
print(f"文件内容: {read_result}")

# 列出目录
dir_result = list_directory.invoke(".")
print(f"目录内容: {dir_result}")


### 3. 数据分析工具

In [ ]:

from langchain_core.tools import tool
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

@tool
def analyze_csv(file_path: str) -> str:
    """分析CSV文件并返回基本统计信息

    Args:
        file_path: CSV文件路径

    Returns:
        数据分析结果
    """
    try:
        # 读取CSV文件
        df = pd.read_csv(file_path)

        # 基本统计
        result = "数据分析结果:\n\n"
        result += f"行数: {len(df)}\n"
        result += f"列数: {len(df.columns)}\n"
        result += f"列名: {', '.join(df.columns)}\n\n"

        # 数值列的统计信息
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            result += "数值列统计:\n"
            for col in numeric_cols:
                stats = df[col].describe()
                result += f"\n{col}:\n"
                result += f"  平均值: {stats['mean']:.2f}\n"
                result += f"  中位数: {stats['50%']:.2f}\n"
                result += f"  最小值: {stats['min']:.2f}\n"
                result += f"  最大值: {stats['max']:.2f}\n"
                result += f"  标准差: {stats['std']:.2f}\n"

        # 缺失值信息
        missing = df.isnull().sum()
        if missing.sum() > 0:
            result += "\n缺失值统计:\n"
            for col, count in missing.items():
                if count > 0:
                    result += f"  {col}: {count} ({count/len(df)*100:.1f}%)\n"

        return result
    except Exception as e:
        return f"分析CSV文件出错: {str(e)}"

@tool
def plot_data(file_path: str, x_column: str, y_column: str, plot_type: str = "line") -> str:
    """绘制数据图表

    Args:
        file_path: CSV文件路径
        x_column: X轴列名
        y_column: Y轴列名
        plot_type: 图表类型，可选 "line", "bar", "scatter"

    Returns:
        图表描述或错误信息
    """
    try:
        # 读取CSV文件
        df = pd.read_csv(file_path)

        # 创建图表
        plt.figure(figsize=(10, 6))

        if plot_type == "line":
            df.plot(x=x_column, y=y_column, kind='line')
        elif plot_type == "bar":
            df.plot(x=x_column, y=y_column, kind='bar')
        elif plot_type == "scatter":
            df.plot(x=x_column, y=y_column, kind='scatter')
        else:
            return f"不支持的图表类型: {plot_type}"

        plt.title(f"{y_column} vs {x_column}")
        plt.xlabel(x_column)
        plt.ylabel(y_column)
        plt.grid(True)

        # 保存图表到文件
        output_file = "plot_output.png"
        plt.savefig(output_file)
        plt.close()

        return f"图表已生成并保存为 {output_file}"
    except Exception as e:
        return f"绘制图表出错: {str(e)}"

# 使用示例 - 需要先创建一个示例CSV文件
def create_sample_csv():
    data = {
        'Year': [2018, 2019, 2020, 2021, 2022],
        'Sales': [150, 200, 180, 300, 250],
        'Expenses': [130, 150, 170, 200, 220]
    }
    df = pd.DataFrame(data)
    df.to_csv('sample_data.csv', index=False)
    return 'sample_data.csv'

# 创建示例数据
sample_file = create_sample_csv()

# 分析数据
analysis_result = analyze_csv.invoke(sample_file)
print(analysis_result)

# 绘制图表
plot_result = plot_data.invoke({
    "file_path": sample_file,
    "x_column": "Year",
    "y_column": "Sales",
    "plot_type": "line"
})
print(plot_result)


### 4. API 调用工具

In [ ]:
from langchain_core.tools import tool
import requests
import json

@tool
def fetch_weather(city: str, api_key: str = "demo_key") -> str:
    """获取指定城市的天气信息

    Args:
        city: 城市名称
        api_key: API密钥（可选）

    Returns:
        天气信息
    """
    try:
        # 注意：这是一个模拟API调用，实际应用中应使用真实的天气API
        # 例如 OpenWeatherMap API

        # 模拟API响应
        weather_data = {
            "city": city,
            "temperature": 23,
            "condition": "晴朗",
            "humidity": 65,
            "wind_speed": 10
        }

        return f"{city}的天气：{weather_data['condition']}，温度{weather_data['temperature']}°C，湿度{weather_data['humidity']}%，风速{weather_data['wind_speed']}km/h"
    except Exception as e:
        return f"获取天气信息出错: {str(e)}"

@tool
def translate_text(text: str, target_language: str, source_language: str = "auto") -> str:
    """翻译文本到指定语言

    Args:
        text: 要翻译的文本
        target_language: 目标语言代码（如'en'、'zh'、'ja'等）
        source_language: 源语言代码，默认为自动检测

    Returns:
        翻译后的文本
    """
    try:
        # 注意：这是一个模拟API调用，实际应用中应使用真实的翻译API
        # 例如 Google Translate API

        # 简单模拟翻译结果
        translations = {
            "en": {
                "你好": "Hello",
                "世界": "World"
            },
            "ja": {
                "你好": "こんにちは",
                "世界": "世界"
            },
            "fr": {
                "你好": "Bonjour",
                "世界": "Monde"
            }
        }

        # 检查是否支持目标语言
        if target_language not in translations:
            return f"不支持的目标语言: {target_language}"

        # 检查是否有该文本的翻译
        if text in translations[target_language]:
            translated = translations[target_language][text]
        else:
            # 模拟翻译
            translated = f"[{text} 的 {target_language} 翻译]"

        return f"原文 ({source_language}): {text}\n翻译 ({target_language}): {translated}"
    except Exception as e:
        return f"翻译文本出错: {str(e)}"

# 使用示例
weather_result = fetch_weather.invoke("北京")
print(weather_result)

translation_result = translate_text.invoke({
    "text": "你好",
    "target_language": "en"
})
print(translation_result)


### 5. 代码执行工具

In [ ]:
from langchain_core.tools import tool
import subprocess
import sys

@tool
def execute_python(code: str) -> str:
    """执行Python代码并返回结果

    Args:
        code: 要执行的Python代码

    Returns:
        代码执行结果
    """
    try:
        # 创建一个安全的本地环境
        local_env = {}

        # 捕获标准输出
        old_stdout = sys.stdout
        sys.stdout = mystdout = io.StringIO()

        try:
            # 执行代码
            exec(code, local_env)
            output = mystdout.getvalue()
            return output if output.strip() else "代码执行成功，无输出"
        finally:
            # 恢复标准输出
            sys.stdout = old_stdout
    except Exception as e:
        return f"代码执行出错: {str(e)}"

@tool
def execute_shell(command: str) -> str:
    """执行Shell命令并返回结果

    Args:
        command: 要执行的Shell命令

    Returns:
        命令执行结果
    """
    try:
        # 注意：在生产环境中应该限制可执行的命令，以防安全风险
        allowed_commands = ["ls", "dir", "echo", "pwd", "date", "whoami"]

        # 检查命令是否在允许列表中
        command_parts = command.split()
        if not command_parts or command_parts[0] not in allowed_commands:
            return f"安全限制：不允许执行命令 '{command_parts[0] if command_parts else command}'"

        # 执行命令
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            timeout=5  # 设置超时时间
        )

        if result.returncode == 0:
            return result.stdout
        else:
            return f"命令执行出错 (返回码 {result.returncode}):\n{result.stderr}"
    except subprocess.TimeoutExpired:
        return "命令执行超时"
    except Exception as e:
        return f"执行Shell命令出错: {str(e)}"

# 使用示例
python_result = execute_python.invoke("""
print("Hello, World!")
for i in range(5):
    print(f"数字: {i}")
""")
print("Python代码执行结果:")
print(python_result)

shell_result = execute_shell.invoke("echo 当前日期是$(date)")
print("\nShell命令执行结果:")
print(shell_result)


## 工具链与 LCEL 集成

In [ ]:

from langchain_community.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import json

# 定义工具
@tool
def get_stock_price(symbol: str) -> str:
    """获取股票价格

    Args:
        symbol: 股票代码

    Returns:
        股票价格信息
    """
    # 模拟股票数据
    stocks = {
        "AAPL": {"price": 173.50, "change": 2.30},
        "MSFT": {"price": 380.20, "change": -0.50},
        "GOOG": {"price": 142.75, "change": 1.25},
        "AMZN": {"price": 178.30, "change": 3.20},
    }

    if symbol in stocks:
        data = stocks[symbol]
        return f"{symbol}当前价格: ${data['price']}, 变动: ${data['change']}"
    else:
        return f"未找到股票 {symbol} 的信息"

@tool
def calculate_investment(initial_amount: float, annual_return: float, years: int) -> str:
    """计算投资回报

    Args:
        initial_amount: 初始投资金额
        annual_return: 年回报率（小数形式，如0.08表示8%）
        years: 投资年限

    Returns:
        投资回报计算结果
    """
    final_amount = initial_amount * (1 + annual_return) ** years
    profit = final_amount - initial_amount

    return json.dumps({
        "initial_investment": initial_amount,
        "annual_return_rate": f"{annual_return * 100}%",
        "investment_period": f"{years}年",
        "final_amount": round(final_amount, 2),
        "total_profit": round(profit, 2),
        "profit_percentage": f"{round((profit / initial_amount) * 100, 2)}%"
    }, ensure_ascii=False)

# 创建聊天模型
chat_model = ChatOllama(model="qwen2.5:3b")

# 绑定工具到模型
chat_with_tools = chat_model.bind_tools([get_stock_price, calculate_investment])

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个投资顾问助手，可以提供股票信息和投资计算。使用提供的工具来回答用户问题。"),
    ("human", "{input}")
])

# 创建LCEL链
chain = (
    {"input": RunnablePassthrough()}
    | prompt
    | chat_with_tools
    | StrOutputParser()
)

# 使用链
def run_investment_assistant():
    print("投资顾问助手 (输入'退出'结束)")
    print("-" * 50)

    while True:
        user_input = input("\n请输入您的问题: ")
        if user_input.lower() in ['退出', 'exit', 'quit']:
            print("谢谢使用！再见！")
            break

        try:
            response = chain.invoke(user_input)
            print("\n助手回答:", response)
        except Exception as e:
            print(f"\n发生错误: {str(e)}")

# 运行助手
if __name__ == "__main__":
    run_investment_assistant()


## 高级工具应用


### 1. 多模态工具

In [ ]:

from langchain_core.tools import tool
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
import base64
from PIL import Image
import io
import matplotlib.pyplot as plt
import numpy as np

@tool
def generate_chart(chart_type: str, data: list, title: str = "图表") -> str:
    """生成数据可视化图表

    Args:
        chart_type: 图表类型，支持 "bar", "line", "pie"
        data: 数据列表
        title: 图表标题

    Returns:
        图表生成结果描述
    """
    try:
        plt.figure(figsize=(10, 6))

        if chart_type == "bar":
            labels = [f"项目{i+1}" for i in range(len(data))]
            plt.bar(labels, data)
        elif chart_type == "line":
            plt.plot(data)
            plt.xticks(range(len(data)), [f"点{i+1}" for i in range(len(data))])
        elif chart_type == "pie":
            labels = [f"部分{i+1}" for i in range(len(data))]
            plt.pie(data, labels=labels, autopct='%1.1f%%')
        else:
            return f"不支持的图表类型: {chart_type}"

        plt.title(title)
        plt.grid(True)

        # 保存图表到文件
        output_file = f"{chart_type}_chart.png"
        plt.savefig(output_file)
        plt.close()

        return f"图表已生成并保存为 {output_file}"
    except Exception as e:
        return f"生成图表出错: {str(e)}"

# 使用示例
chart_result = generate_chart.invoke({
    "chart_type": "bar",
    "data": [12, 19, 3, 5, 2, 3],
    "title": "示例柱状图"
})
print(chart_result)

# 如果有支持多模态的模型，可以这样使用
def multimodal_example():
    try:
        # 注意：这需要支持多模态的模型
        model = ChatOllama(model="llava:7b")

        # 生成图表
        generate_chart.invoke({
            "chart_type": "pie",
            "data": [35, 25, 20, 15, 5],
            "title": "收入分布"
        })

        # 读取图表图像
        with open("pie_chart.png", "rb") as img_file:
            img_data = base64.b64encode(img_file.read()).decode()

        # 创建带图像的消息
        message = HumanMessage(
            content=[
                {"type": "text", "text": "请分析这个收入分布图并给出建议"},
                {"type": "image_url", "image_url": f"data:image/png;base64,{img_data}"}
            ]
        )

        # 调用模型
        response = model.invoke([message])
        print("")
    except:
        pass
